In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score
from joblib import dump, load
from sklearn.linear_model import LogisticRegression

# Checking the classification Power of the synth data created from data

Data Loading

In [7]:
df_train = pd.read_csv("../data/interim/split/sorted_train_data_mm.csv")

synthetic_data_paths = [
    "../data/interim/split/sorted_train_data_mm.csv",
    "../data/interim/split/sorted_test_data_mm.csv",
    "../data/interim/synthdata_from_data.csv",
    "../data/interim/synthdata_from_sample.csv",
    "../data/interim/generated/cgan2500.csv",
    "../data/interim/generated/cgan5000.csv",
    "../data/interim/generated/cgan10000.csv",
    "../data/interim/generated/CGANEPOCH1.csv",
    "../data/interim/generated/CGANEPOCH2.csv",
]

file_names = [path.split("/")[-1].split(".")[0] for path in synthetic_data_paths]

In [8]:
synth_data_dict = {}

for path, file_name in zip(synthetic_data_paths, file_names):
    synth_data_dict[file_name] = {}
    synth_data = pd.read_csv(path)
    synth_data_dict[file_name]["X"] = synth_data.values[:, :-1]
    synth_data_dict[file_name]["y"] = synth_data.values[:, -1]

Train Random forest and Logistic Regression on original data

In [51]:
# # load train_data
# X = synth_data_dict["sorted_train_data_mm"]["X"]
# y = synth_data_dict["sorted_train_data_mm"]["y"]

# # Random Forest
# random_forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
# random_forest.fit(X, y)
# dump(random_forest, "../models/Classifier/random_forest_model.joblib")

# # Logistic Regression
# logistic_regression = LogisticRegression()
# logistic_regression.fit(X, y)
# dump(logistic_regression, "../models/Classifier/logistic_regression.joblib")

['../models/Classifier/logistic_regression.joblib']

# load the models

In [6]:
random_forest = load("../models/Classifier/random_forest_model.joblib")
logistic_regression = load("../models/Classifier/logistic_regression.joblib")

In [58]:
def evaluate_model_performance_random_forest(model, X, y, name):
    
    # Predictions on training and testing data
    y_pred = model.predict(X)

    # Accuracy scores
    accuracy = accuracy_score(y, y_pred)

    # Recall scores
    recall = recall_score(y, y_pred)

    # F1 scores
    f1 = f1_score(y, y_pred)

    # Confusion Matrix
    conf_matrix = confusion_matrix(y, y_pred)

    # Create new row
    row =  [model, name, accuracy, recall, f1, conf_matrix]
 
    
    return row

In [59]:
df_evaluation = pd.DataFrame(
    columns=[
        "Model",
        "Data Set",
        "Accuracy",
        "Recall",
        "F1 Score",
        "Confusion Matrix"
    ]
)

for key in synth_data_dict:
    X = synth_data_dict[key]["X"]
    y = synth_data_dict[key]["y"]
    evaluation_result = evaluate_model_performance_random_forest(random_forest, X, y, key)
    df_evaluation.loc[len(df_evaluation)] = evaluation_result


for key in synth_data_dict:
    X = synth_data_dict[key]["X"]
    y = synth_data_dict[key]["y"]
    evaluation_result = evaluate_model_performance_random_forest(
        logistic_regression, X, y, key
    )
    df_evaluation.loc[len(df_evaluation)] = evaluation_result

In [60]:
df_evaluation

,Model,Data Set,Accuracy,Recall,F1 Score,Confusion Matrix
0,"(DecisionTreeClassifier(max_depth=10, max_feat...",sorted_train_data_mm,0.999679,0.820225,0.901235,"[[199008, 0], [64, 292]]"
1,"(DecisionTreeClassifier(max_depth=10, max_feat...",sorted_test_data_mm,0.999544,0.771739,0.845238,"[[56865, 5], [21, 71]]"
2,"(DecisionTreeClassifier(max_depth=10, max_feat...",synthdata_from_data,0.998912,0.881720,0.841026,"[[28367, 20], [11, 82]]"
3,"(DecisionTreeClassifier(max_depth=10, max_feat...",synthdata_from_sample,0.643680,0.593742,0.212935,"[[29774, 16167], [1649, 2410]]"
4,"(DecisionTreeClassifier(max_depth=10, max_feat...",cgan2500,0.998754,0.228261,0.371681,"[[56870, 0], [71, 21]]"
5,"(DecisionTreeClassifier(max_depth=10, max_feat...",cgan5000,0.999877,0.945652,0.961326,"[[56868, 2], [5, 87]]"
6,"(DecisionTreeClassifier(max_depth=10, max_feat...",cgan10000,0.999930,0.956522,0.977778,"[[56870, 0], [4, 88]]"
7,"(DecisionTreeClassifier(max_depth=10, max_feat...",CGANEPOCH1,0.999368,0.608696,0.756757,"[[56870, 0], [36, 56]]"
8,"(DecisionTreeClassifier(max_depth=10, max_feat...",CGANEPOCH2,0.999052,0.413043,0.584615,"[[56870, 0], [54, 38]]"
9,LogisticRegression(),sorted_train_data_mm,0.999022,0.528090,0.658494,"[[198981, 27], [168, 188]]"
